In [5]:
import collections
import helper
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from tqdm import tqdm

In [6]:
pip install keras-self-attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Conv1D, MaxPool1D, Dense, Flatten, Dropout, AveragePooling2D, LSTM, TimeDistributed, Attention
import matplotlib.pyplot as plt
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import keras
from keras import models, layers
from keras import backend
from sklearn.metrics import f1_score,recall_score,precision_score, confusion_matrix
import keras
from keras_self_attention import SeqSelfAttention
import collections
import numpy as np
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.models import Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SLU Semesters/SLU 3rd Semester/NLP/Fifth Competition/

%ls

In [9]:
# def read_data(file_name):
#   data=[]
#   with open(file_name, encoding='utf8') as f:
#     for line in f:
#       line=line.strip()
#       data.append(line)
#   size = len(data)
#   idx_list = [idx + 1 for idx, val in
#             enumerate(data) if val == '</s>']
#   res = [data[i: j] for i, j in
#         zip([0] + idx_list, idx_list + 
#         ([size] if idx_list[-1] != size else []))]
#   return res

In [10]:
def read_data(file_name):
    all_data = []
    descript = 'Reading ' + file_name

    f = open(file_name, 'r', encoding='utf-8')
    full_text = f.read()

    cur_sent = []

    for line in tqdm(full_text.split('\n'), desc=descript):
        if line == '<s>':
            cur_sent = []
            continue
        if line in '()':
            continue
        if line == '</s>':
            all_data.append(cur_sent)
            continue
        else:
            cur_sent.append(line.lower())

    return all_data

In [11]:
%ls

models/  Neural_Machine_Translation.ipynb  test-05/  train-05/


In [ ]:
# source = read_data('train-05/train-source.txt')
# target = read_data('train-05/train-target.txt')

source = read_data('test-05/test-source.txt')
target = read_data('test-05/test-target.txt')

source

In [13]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

In [14]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

In [15]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [23]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

preproc_source, preproc_target, source_tokenizer, target_tokenizer =\
    preprocess(source, target)
    
max_source_length = preproc_source.shape[1]
max_target_length = preproc_target.shape[1]
source_vocab_size = len(source_tokenizer.word_index)
target_vocab_size = len(target_tokenizer.word_index)

print('Data Preprocessed')
print("Max source sentence length:", max_source_length)
print("Max target sentence length:", max_target_length)
print("Source vocabulary size:", source_vocab_size)
print("Target vocabulary size:", target_vocab_size)

Data Preprocessed
Max source sentence length: 97
Max target sentence length: 92
Source vocabulary size: 3215
Target vocabulary size: 2997


In [17]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ''

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [14]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model

In [15]:
# def build_model(input_shape, output_sequence_length, source_vocab_size, target_vocab_size):

#     model = Sequential()
#     model.add(Embedding(input_dim=source_vocab_size,output_dim=256,input_length=input_shape[1]))
#     model.add(Bidirectional(LSTM(256, activation="tanh",return_sequences=True)))
#     model.add(SeqSelfAttention(attention_activation='softmax'))
#     # model.add((LSTM(512, activation = 'relu')))
#     model.add(Dense(512, activation = 'relu'))
#     model.add(Dense(128, activation = 'relu'))
#     model.add(Dense(64, activation = 'relu'))
#     model.add((Dense(target_vocab_size,activation='softmax')))
#     learning_rate = 0.005

#     model.compile(loss = sparse_categorical_crossentropy,
#                  optimizer = Adam(learning_rate),
#                  metrics = ['accuracy'])

#     return model

In [30]:
tmp_x = pad(preproc_source, max_target_length)
tmp_x = tmp_x.reshape((-1, preproc_target.shape[-2], 1))

In [17]:
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_target_length,
    source_vocab_size,
    target_vocab_size)

simple_rnn_model.fit(tmp_x, preproc_target, batch_size=32, epochs=1, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], target_tokenizer))

1/1 [==============================] - 0s 333ms/step
" a a a a a a a a a a a . .                                                                                                                                                                                                               


In [18]:
# # source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

# # Train RNN

# # Train the neural network
# simple_rnn_model = simple_model(
#     tmp_x.shape,
#     max_target_length,
#     source_vocab_size,
#     target_vocab_size)

# simple_rnn_model.fit(tmp_x, preproc_target, batch_size=32, epochs=1, validation_split=0.2)

# # Print prediction(s)
# print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], target_tokenizer))

In [19]:
simple_rnn_model.save('./models/simple_rnn_model.h5')

In [20]:
%ls ./models/

simple_rnn_model.h5


In [31]:
from keras.models import load_model
loaded_rnn = load_model("./models/simple_rnn_model.h5")

In [22]:
# # Train RNN for test

# preproc_source_test, preproc_target_test, source_test_tokenizer, target_test_tokenizer =\
#     preprocess(source_test, target_test)
    
# max_source_test_length = preproc_source_test.shape[1]
# max_target_test_length = preproc_target_test.shape[1]
# source_test_vocab_size = len(source_test_tokenizer.word_index)
# target_test_vocab_size = len(target_test_tokenizer.word_index)

# tmp_x_test = pad(preproc_source_test, max_target_test_length)
# tmp_x_test = tmp_x_test.reshape((-1, preproc_target_test.shape[-2], 1))

In [ ]:
loaded_rnn.predict(tmp_x[:1])[0]

In [ ]:
print(logits_to_text(loaded_rnn.predict(tmp_x[:1])[0], target_tokenizer))

In [25]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd
from keras.models import Sequential

def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(french_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

In [ ]:
# Train embedding model

tmp_x = pad(preproc_source, max_target_length)
tmp_x = tmp_x.reshape((-1, preproc_target.shape[-2]))


embeded_model = embed_model(
    tmp_x.shape,
    max_target_length,
    source_vocab_size,
    target_vocab_size)

embeded_model.fit(tmp_x, preproc_target, batch_size=32, epochs=1, validation_split=0.2)

print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], target_tokenizer))

In [18]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model

In [20]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd
# Train bi-directional
tmp_x = pad(preproc_source, preproc_target.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_target.shape[-2], 1))

bidi_model = bd_model(
    tmp_x.shape,
    preproc_target.shape[1],
    len(source_tokenizer.word_index)+1,
    len(target_tokenizer.word_index)+1)


bidi_model.fit(tmp_x, preproc_target, batch_size=32, epochs=2, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], target_tokenizer))

Epoch 1/2
1130/1130 [==============================] - 341s 294ms/step - loss: 0.7411 - accuracy: 0.9194 - val_loss: 0.4874 - val_accuracy: 0.9287
Epoch 2/2
1/1 [==============================] - 1s 589ms/step
bhí an a a a a a a a a a a . .                                                                                                                                                                                                               


In [21]:
bidi_model.save('./models/bidire_model.h5')
from keras.models import load_model
loaded_bid = load_model("./models/bidire_model.h5")

In [24]:
print(logits_to_text(loaded_bid.predict(tmp_x[:1])[0], target_tokenizer))

1/1 [==============================] - 1s 556ms/step
bhí an . . . . . . . . . . a a                                                                                                                                                                                                               


In [ ]:
# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model

In [ ]:

# source: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model